In [6]:
# Libraries
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as data
import datetime
import json
from yahoofinancials import YahooFinancials
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import requests
import warnings




asset_dict = {}

# Assets
russell_index = ["^RUI"]
tickers = []

# Set timeframe
start = "2019-03-31"
end = "2019-04-29"

def russell():
    # Pull data from Yahoo! Finance for Russell 1000 Index
    df = data.DataReader(russell_index, data_source="yahoo", start=start, end=end)

    percentage = df.pct_change()
    russellReturn = percentage["Adj Close"]

    return russellReturn


#russell()

def pullTickers():
    exec(open("scrapeETF.py").read())

    list(asset_dict.keys())

    for key in asset_dict.keys():
        tickers.append(key.replace('.','-'))
    return tickers


pullTickers()




Looking for [chromedriver 79.0.3945.36 win32] driver in cache
File found in cache by path [C:\Users\lukes\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]


['AAPL',
 'MSFT',
 'AMZN',
 'FB',
 'BRK-B',
 'JPM',
 'GOOG',
 'GOOGL',
 'JNJ',
 'V',
 'PG',
 'XOM',
 'BAC',
 'T',
 'UNH',
 'MA',
 'INTC',
 'VZ',
 'DIS',
 'HD',
 'MRK',
 'CVX',
 'PFE',
 'KO',
 'CSCO']

In [110]:
yahoo_financials = YahooFinancials('AAPL')
print(yahoo_financials.get_key_statistics_data()['AAPL']['sharesOutstanding'])

4384030208


In [102]:
yahoo_financials = YahooFinancials('AMZN')
print(yahoo_financials.get_financial_stmts('quarterly', 'balance')['balanceSheetHistoryQuarterly']['AMZN'][2])

{'2019-03-31': {'capitalSurplus': 28059000000, 'totalLiab': 129692000000, 'totalStockholderEquity': 48410000000, 'otherCurrentLiab': 7298000000, 'totalAssets': 178102000000, 'commonStock': 5000000, 'otherCurrentAssets': 359000000, 'retainedEarnings': 23193000000, 'otherLiab': 9400000000, 'goodWill': 14708000000, 'treasuryStock': -2847000000, 'otherAssets': 12189000000, 'cash': 23115000000, 'totalCurrentLiabilities': 63695000000, 'shortLongTermDebt': 1608000000, 'otherStockholderEquity': -1010000000, 'propertyPlantEquipment': 81180000000, 'totalCurrentAssets': 69431000000, 'longTermInvestments': 594000000, 'netTangibleAssets': 33702000000, 'shortTermInvestments': 13905000000, 'netReceivables': 15620000000, 'longTermDebt': 23322000000, 'inventory': 16432000000, 'accountsPayable': 31809000000}}


In [31]:
yahoo_financials = YahooFinancials('MSFT')
print(yahoo_financials.get_financial_stmts('quarterly', 'balance')['balanceSheetHistoryQuarterly']['MSFT'][2]['2019-03-31']['commonStock'])

77791000000


In [105]:
replicatedWeights = {}


def pullFundamentals():

    for i in tickers:
        yahoo_financials = YahooFinancials(i)
        
        replicatedWeights[i] = price / marketCap
        
pullFundamentals()

{'AAPL': 41454000000}
{'AAPL': 41454000000, 'MSFT': 73996000000}
{'AAPL': 41454000000, 'MSFT': 73996000000, 'AMZN': 21360000000}


KeyboardInterrupt: 

In [85]:
def buildPortfolio():

    df = data.DataReader(tickers, data_source="yahoo", start=start, end=end)

    return(df['Adj Close'])
buildPortfolio()











Symbols,AAPL,AMZN,BAC,BRK-B,CSCO,CVX,DIS,FB,GOOG,GOOGL,...,MA,MRK,MSFT,PFE,PG,T,UNH,V,VZ,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-01,189.217209,1814.189941,28.047619,205.000000,53.490116,121.268890,111.151596,168.699997,1194.430054,1198.979980,...,237.820801,81.641251,117.786697,41.667114,101.002380,30.153536,242.378220,156.530075,56.692638,78.860390
2019-04-02,191.967804,1813.979980,28.371925,203.880005,53.791718,121.259186,110.608238,174.199997,1200.489990,1205.540039,...,238.029709,81.553032,117.954941,41.735195,101.109596,29.851526,241.262772,157.047653,56.155357,78.522659
2019-04-03,193.283737,1820.699951,28.342443,203.669998,54.122505,120.249741,111.161469,173.539993,1205.920044,1210.810059,...,237.701401,81.523636,118.726852,41.667114,101.051117,30.078032,242.674362,157.724518,56.481564,78.059517
2019-04-04,193.620117,1818.859985,28.647093,204.100006,53.985435,121.133011,113.364548,176.020004,1215.000000,1219.449951,...,234.846146,80.220116,118.123177,41.569851,100.797737,30.181849,244.204407,156.908310,56.596699,79.169144
2019-04-05,194.916260,1837.280029,28.578300,205.330002,54.053970,122.705406,113.611526,175.720001,1207.150024,1211.449951,...,235.413223,79.534058,118.647690,41.813004,101.012131,30.531042,245.576508,156.918259,56.692638,79.593689
2019-04-08,197.983475,1849.859985,28.666748,205.250000,54.328106,122.957764,113.572014,174.929993,1203.839966,1208.280029,...,235.383362,79.338036,118.687271,41.958897,102.298538,30.568792,245.546890,157.017792,56.731018,80.085785
2019-04-09,197.389832,1835.839966,28.391577,204.919998,54.024597,121.851265,115.449074,177.580002,1197.250000,1202.689941,...,235.024689,79.191032,118.043999,41.667114,101.996429,30.444141,245.586380,156.759018,56.607433,79.053352
2019-04-10,198.497971,1847.329956,28.568474,204.520004,54.651196,121.802727,115.745461,177.820007,1202.160034,1206.449951,...,236.170380,79.210632,118.944572,41.560123,101.986694,30.568792,242.861908,157.824051,56.810986,78.696342
2019-04-11,196.845642,1844.069946,28.568474,206.669998,54.435799,122.288040,115.192215,177.509995,1204.619995,1209.589966,...,235.154205,78.250145,119.083122,41.112717,102.084137,30.875631,232.388535,157.127289,56.762520,79.072655
